In [ ]:
from datetime import date, datetime, timedelta
import os
import math

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
transactions_df = pd.read_feather("/content/transactions.feather")
transactions_df.sample(10, random_state=0)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
1359175,1368663,2018-08-21 15:58:32,3812,3882,12.82,12326312,142,0,0
744402,753890,2018-06-18 13:24:19,4163,267,119.25,6787459,78,0,0
743737,753225,2018-06-18 12:29:50,1140,710,33.79,6784190,78,0,0
681649,691137,2018-06-12 04:01:22,3962,1398,24.36,6235282,72,0,0
412664,422152,2018-05-14 22:03:16,1589,3468,21.74,3794596,43,0,0
1275701,1285189,2018-08-13 03:32:52,36,5830,20.62,11590372,134,0,0
821080,830568,2018-06-26 13:30:53,609,8256,24.29,7479053,86,0,0
1667347,1676835,2018-09-22 18:58:10,2168,1544,159.67,15101890,174,0,0
93486,102974,2018-04-11 15:25:15,4904,7764,100.05,919515,10,0,0
896790,906278,2018-07-04 11:33:25,565,2805,10.99,8163205,94,0,0


DATASET FOR THE GIVEN PROJECT-SAMPLE


NUMBER OF FRAUD AND LEGIT TRANSACTIONS


In [ ]:
not_fraud_count, fraud_count = np.bincount(transactions_df["TX_FRAUD"])

total_count = not_fraud_count + fraud_count
print(
    (
        f"Data:\n"
        f"    Total: {total_count}\n"
        f"    Fraud: {fraud_count} ({100 * fraud_count / total_count:.2f}% of total)\n"
    )
)

Data:
    Total: 1744667
    Fraud: 14678 (0.84% of total)



STATISTICAL DIFFERENCE BETWEEN FRAUD AND LEGIT TRANSACTION

In [ ]:
df = pd.concat(
    [
        transactions_df[transactions_df["TX_FRAUD"] == 0].sample(1000, random_state=0),
        transactions_df[transactions_df["TX_FRAUD"] == 1].sample(1000, random_state=0),
    ]
)

fig = px.histogram(
    df,
    title="Transaction count for different amounts",
    x="TX_AMOUNT",
    color="TX_FRAUD",
    marginal="box",
)
fig.update_traces(opacity=0.75)
fig.update_layout(barmode="overlay")
fig.show()

In [ ]:
cleaned_df = pd.DataFrame()

SORTING OUT THE DATA

In [ ]:
cleaned_df["amount"] = transactions_df["TX_AMOUNT"]
cleaned_df["is_fraud"] = transactions_df["TX_FRAUD"]
cleaned_df["is_weekend"] = transactions_df["TX_DATETIME"].dt.weekday >= 5
cleaned_df["is_night"] = transactions_df["TX_DATETIME"].dt.hour <= 6

amount: the amount of the transaction
is_fraud: whether the transaction was fraudulent or not (this is technically the label, not a feature)

is_weekend: whether the transaction was on a weekend or not

is_night: whether the transaction was at night or not (we take "night" to mean late at night: after midnight and before 6am)

In [ ]:
cleaned_df["customer_num_transactions_1_day"] = transactions_df.groupby(
    "CUSTOMER_ID"
).apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("1d", on="TX_DATETIME").count()
)[
    "TX_AMOUNT"
]
cleaned_df["customer_num_transactions_7_day"] = transactions_df.groupby(
    "CUSTOMER_ID"
).apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("7d", on="TX_DATETIME").count()
)[
    "TX_AMOUNT"
]
cleaned_df["customer_num_transactions_30_day"] = transactions_df.groupby(
    "CUSTOMER_ID"
).apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("30d", on="TX_DATETIME").count()
)[
    "TX_AMOUNT"
]

cleaned_df["customer_avg_amount_1_day"] = transactions_df.groupby("CUSTOMER_ID").apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("1d", on="TX_DATETIME").mean()
)["TX_AMOUNT"]
cleaned_df["customer_avg_amount_7_day"] = transactions_df.groupby("CUSTOMER_ID").apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("7d", on="TX_DATETIME").mean()
)["TX_AMOUNT"]
cleaned_df["customer_avg_amount_30_day"] = transactions_df.groupby("CUSTOMER_ID").apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("30d", on="TX_DATETIME").mean()
)["TX_AMOUNT"]

<ipython-input-9-8223b00a9fbf>:3: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)

<ipython-input-9-8223b00a9fbf>:10: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)

<ipython-input-9-8223b00a9fbf>:17: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future

In [ ]:
DAY_DELAY = 7

def get_count_risk_rolling_window(
    terminal_transactions, window_size, delay_period=DAY_DELAY
):
    frauds_in_delay = terminal_transactions.rolling(
        str(delay_period) + "d", on="TX_DATETIME"
    )["TX_FRAUD"].sum()
    transactions_in_delay = terminal_transactions.rolling(
        str(delay_period) + "d", on="TX_DATETIME"
    )["TX_FRAUD"].count()

    frauds_until_window = terminal_transactions.rolling(
        str(delay_period + window_size) + "d", on="TX_DATETIME"
    )["TX_FRAUD"].sum()
    transactions_until_window = terminal_transactions.rolling(
        str(delay_period + window_size) + "d", on="TX_DATETIME"
    )["TX_FRAUD"].count()

    frauds_in_window = frauds_until_window - frauds_in_delay
    transactions_in_window = transactions_until_window - transactions_in_delay

    terminal_transactions["fraud_risk"] = (
        frauds_in_window / transactions_in_window
    ).fillna(0)

    return terminal_transactions


cleaned_df["terminal_num_transactions_1_day"] = transactions_df.groupby(
    "TERMINAL_ID"
).apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("1d", on="TX_DATETIME").count()
)[
    "TX_AMOUNT"
]
cleaned_df["terminal_num_transactions_7_day"] = transactions_df.groupby(
    "TERMINAL_ID"
).apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("7d", on="TX_DATETIME").count()
)[
    "TX_AMOUNT"
]
cleaned_df["terminal_num_transactions_30_day"] = transactions_df.groupby(
    "TERMINAL_ID"
).apply(
    lambda x: x[["TX_DATETIME", "TX_AMOUNT"]].rolling("30d", on="TX_DATETIME").count()
)[
    "TX_AMOUNT"
]

cleaned_df["terminal_fraud_risk_1_day"] = transactions_df.groupby("TERMINAL_ID").apply(
    lambda x: get_count_risk_rolling_window(x, 1, 7)
)["fraud_risk"]
cleaned_df["terminal_fraud_risk_7_day"] = transactions_df.groupby("TERMINAL_ID").apply(
    lambda x: get_count_risk_rolling_window(x, 7, 7)
)["fraud_risk"]
cleaned_df["terminal_fraud_risk_30_day"] = transactions_df.groupby("TERMINAL_ID").apply(
    lambda x: get_count_risk_rolling_window(x, 30, 7)
)["fraud_risk"]

<ipython-input-10-f8c4aaf5fb1d>:32: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)

<ipython-input-10-f8c4aaf5fb1d>:39: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)

<ipython-input-10-f8c4aaf5fb1d>:46: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the fu

In [ ]:
cleaned_df["day"] = transactions_df["TX_TIME_DAYS"]
cleaned_df["datetime"] = transactions_df["TX_DATETIME"]
cleaned_df["customer_id"] = transactions_df["CUSTOMER_ID"]
cleaned_df["id"] = transactions_df["TRANSACTION_ID"]

In [ ]:
pd.concat(
    # show some fraudulent and non-fraudulent transactions
    [
        cleaned_df[cleaned_df["is_fraud"] == 1].sample(5, random_state=0),
        cleaned_df[cleaned_df["is_fraud"] == 0].sample(5, random_state=0),
    ]
).sample(10, random_state=0)

,amount,is_fraud,is_weekend,is_night,customer_num_transactions_1_day,customer_num_transactions_7_day,customer_num_transactions_30_day,customer_avg_amount_1_day,customer_avg_amount_7_day,customer_avg_amount_30_day,terminal_num_transactions_1_day,terminal_num_transactions_7_day,terminal_num_transactions_30_day,terminal_fraud_risk_1_day,terminal_fraud_risk_7_day,terminal_fraud_risk_30_day,day,datetime,customer_id,id
764833,40.66,1,False,False,3.0,11.0,43.0,43.276667,38.531818,38.359302,1.0,10.0,27.0,1.0,1.0,0.347826,80,2018-06-20 15:11:12,2955,774321
1109119,54.77,0,False,False,5.0,16.0,72.0,58.276000,67.723125,83.303333,4.0,10.0,31.0,0.0,0.0,0.000000,116,2018-07-26 14:32:29,376,1118607
408126,26.29,1,False,False,5.0,25.0,106.0,18.080000,19.809600,19.115566,2.0,12.0,36.0,0.0,0.0,0.000000,43,2018-05-14 12:11:16,1262,417614
1437242,132.03,0,False,False,4.0,18.0,64.0,70.092500,55.609444,57.838750,2.0,4.0,34.0,0.0,0.0,0.000000,150,2018-08-29 20:25:14,3463,1446730
1073063,5.15,1,True,False,5.0,28.0,101.0,6.190000,7.026071,7.142376,1.0,8.0,26.0,0.0,1.0,0.272727,112,2018-07-22 18:50:40,935,1082551
1027460,48.39,0,False,True,5.0,23.0,90.0,40.238000,53.501304,57.711111,1.0,7.0,27.0,0.0,0.0,0.000000,108,2018-07-18 06:12:32,2965,1036948
1340733,69.47,0,True,False,3.0,19.0,60.0,72.186667,63.241579,59.846333,2.0,3.0,28.0,0.0,0.0,0.000000,140,2018-08-19 17:31:46,1973,1350221
1416564,54.42,1,False,False,5.0,31.0,124.0,71.286000,62.111935,60.428306,1.0,14.0,48.0,0.0,0.0,0.000000,148,2018-08-27 15:48:02,1310,1426052
498910,29.27,1,False,False,4.0,16.0,72.0,29.747500,33.244375,31.766944,2.0,4.0,25.0,1.0,1.0,0.541667,52,2018-05-23 20:54:13,1949,508398
1741939,87.12,0,True,False,4.0,17.0,49.0,71.335000,65.998235,72.604082,2.0,8.0,26.0,0.0,0.0,0.000000,182,2018-09-30 15:09:42,734,1751427


SLICING AND SEPERATING THE DATA TO SHOW SOME FRADULENT AND LEGIT TRANSACTIONS


In [ ]:
def get_train_test_set(
    df,
    start_date_training,
    delta_train=7,
    delta_delay=DAY_DELAY,
    delta_test=7,
    random_state=0,
):

    # Get the training set data
    train_df = df[
        (df["datetime"] >= start_date_training)
        & (df["datetime"] < start_date_training + timedelta(days=delta_train))
    ]

    # Get the test set data
    test_df = []

    # Note: Cards known to be compromised after the delay period are removed from the test set
    # That is, for each test day, all frauds known at (test_day-delay_period) are removed

    # First, get known defrauded customers from the training set
    known_defrauded_customers = set(train_df[train_df["is_fraud"] == 1]["customer_id"])

    # Get the relative starting day of training set (easier than TX_DATETIME to collect test data)
    start_tx_time_days_training = train_df["day"].min()

    # Then, for each day of the test set
    for day in range(delta_test):

        # Get test data for that day
        test_df_day = df[
            df["day"] == start_tx_time_days_training + delta_train + delta_delay + day
        ]

        # Compromised cards from that test day, minus the delay period, are added to the pool of known defrauded customers
        test_df_day_delay_period = df[
            df["day"] == start_tx_time_days_training + delta_train + day - 1
        ]

        new_defrauded_customers = set(
            test_df_day_delay_period[test_df_day_delay_period["is_fraud"] == 1][
                "customer_id"
            ]
        )
        known_defrauded_customers = known_defrauded_customers.union(
            new_defrauded_customers
        )

        test_df_day = test_df_day[
            ~test_df_day["customer_id"].isin(known_defrauded_customers)
        ]

        test_df.append(test_df_day)

    test_df = pd.concat(test_df)

    # Sort data sets by ascending order of transaction ID
    train_df = train_df.sort_values("id")
    test_df = test_df.sort_values("id")

    return (train_df, test_df)


train_df, test_df = get_train_test_set(
    cleaned_df, datetime(2018, 7, 25), delta_train=21
)
train_df, val_df = get_train_test_set(train_df, datetime(2018, 7, 25))

We need to split the dataset up into train, validation, and test sets.

The training set is used for training the model and is not used to evaluate it.
The validation set is used to track metrics while training but is not used for training.
The test set is not used at all during the training process but used at the end to see how the model generalizes to new data.

In [ ]:
label_columns = ["is_fraud"]
feature_columns = [
    "amount",
    "is_weekend",
    "is_night",
    "customer_num_transactions_1_day",
    "customer_num_transactions_7_day",
    "customer_num_transactions_30_day",
    "customer_avg_amount_1_day",
    "customer_avg_amount_7_day",
    "customer_avg_amount_30_day",
    "terminal_num_transactions_1_day",
    "terminal_num_transactions_7_day",
    "terminal_num_transactions_30_day",
    "terminal_fraud_risk_1_day",
    "terminal_fraud_risk_7_day",
    "terminal_fraud_risk_30_day",
]

train_labels = np.array(train_df[label_columns])
val_labels = np.array(val_df[label_columns])
test_labels = np.array(test_df[label_columns])

train_features = np.array(train_df[feature_columns])
val_features = np.array(val_df[feature_columns])
test_features = np.array(test_df[feature_columns])

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (67240, 1)
Validation labels shape: (58264, 1)
Test labels shape: (50321, 1)
Training features shape: (67240, 15)
Validation features shape: (58264, 15)
Test features shape: (50321, 15)


**The model**
Our model needs to take each row of features and make a prediction of whether a transaction is fraudulent or not.

A simple feed-forward neural network is a good fit for this. It performs a multitude of mathematical calculations on each input row. The parameters of each of these calculations are what end up getting tuned during training.

This model will be built in Keras, which is a user-friendly deep learning API on top of TensorFlow.

**Imbalanced Data**
We are working with imbalanced data. Less than 1% of our dataset contains fraudulent transactions. This makes naive techniques for learning problematic. A model that predicts everything as legitimate would be correct over 99% of the time, but that would be a useless model.

One method to accommodate for this is class weighting. This means penalizing misclassifications of one class (in this case, fraudulent transactions) than the other. Below, we calculate the weights for each class that we will pass to Keras. The weights make the fraudulent labels 120 times "more important" than the non-fraudulent labels.

In [ ]:
weight_for_not_fraud = (1.0 / not_fraud_count) * total_count / 2.0
weight_for_fraud = (1.0 / fraud_count) * total_count / 2.0

class_weight = {0: weight_for_not_fraud, 1: weight_for_fraud}

class_weight

{0: 0.5042422235054674, 1: 59.431359858291316}

In [ ]:
output_bias = tf.keras.initializers.Constant(np.log([fraud_count / not_fraud_count]))

model = keras.Sequential(
    [
        keras.layers.Dense(
            500, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(
            500, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation="sigmoid", bias_initializer=output_bias),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(name="auc"),
        keras.metrics.AUC(name="prc", curve="PR"),
    ],
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               8000      
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dropout (Dropout)           (None, 500)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 259001 (1011.72 KB)
Trainable params: 259001 (1011.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
BATCH_SIZE = 64

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_prc", verbose=1, patience=10, mode="max", restore_best_weights=True
)

training_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=40,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels),
    class_weight=class_weight,
)

Epoch 1/40
1051/1051 [==============================] - 19s 15ms/step - loss: 0.4803 - precision: 0.0661 - recall: 0.7441 - auc: 0.8569 - prc: 0.4054 - val_loss: 0.2195 - val_precision: 0.0849 - val_recall: 0.7013 - val_auc: 0.8782 - val_prc: 0.5055
Epoch 2/40
1051/1051 [==============================] - 10s 9ms/step - loss: 0.3663 - precision: 0.0886 - recall: 0.7843 - auc: 0.9020 - prc: 0.4902 - val_loss: 0.1932 - val_precision: 0.1110 - val_recall: 0.6961 - val_auc: 0.8525 - val_prc: 0.5256
Epoch 3/40
1051/1051 [==============================] - 11s 10ms/step - loss: 0.3618 - precision: 0.0908 - recall: 0.7910 - auc: 0.9033 - prc: 0.5272 - val_loss: 0.5196 - val_precision: 0.0239 - val_recall: 0.7636 - val_auc: 0.8463 - val_prc: 0.4240
Epoch 4/40
1051/1051 [==============================] - 10s 9ms/step - loss: 0.3376 - precision: 0.1006 - recall: 0.8043 - auc: 0.9153 - prc: 0.5331 - val_loss: 0.3995 - val_precision: 0.0402 - val_recall: 0.7740 - val_auc: 0.8726 - val_prc: 0.4196
Ep

TRAINING THE MODEL

In [ ]:
res = []

metrics_to_plot = [
    ("loss", "Loss"),
    ("precision", "Precision"),
    ("recall", "Recall"),
    ("auc", "Area under ROC curve"),
    ("prc", "Area under PR curve"),
]
fig = make_subplots(rows=len(metrics_to_plot), cols=1)

for metric, name in metrics_to_plot:
    fig = go.Figure(
        data=[
            go.Scatter(
                x=training_history.epoch,
                y=training_history.history[metric],
                mode="lines",
                name="Training",
            ),
            go.Scatter(
                x=training_history.epoch,
                y=training_history.history["val_" + metric],
                mode="lines",
                line={"dash": "dash"},
                name="Validation",
            ),
        ]
    )
    fig.update_yaxes(title=name)
    fig.update_xaxes(title="Epoch")

    if (metric, name) == metrics_to_plot[0]:
        fig.update_layout(
            height=250, title="Training history", margin={"b": 0, "t": 50}
        )
    else:
        fig.update_layout(height=200, margin={"b": 0, "t": 0})
    fig.show()


Above is a plot of each of the metrics we are tracking over the course of the training.

**MODEL TRAINING**

The model outputs a number between 0 and 1. Although it isn't technically true, it helps to think of this as the probability that a transaction is fraudulent. A histogram of the outputs of predictions on the training set is below. The correct label is also shown (0 for legitimate, 1 for fraudulent). The amount of samples for each label is slightly biased to make the scales similar.

In [ ]:
train_predictions = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions = model.predict(test_features, batch_size=BATCH_SIZE)

predictions_df = pd.DataFrame(
    {"Prediction": train_predictions.ravel(), "Label": train_labels.ravel()}
)
predictions_df = pd.concat(
    [
        predictions_df[predictions_df["Label"] == 0].sample(5000, random_state=0),
        predictions_df[predictions_df["Label"] == 1].sample(500, random_state=0),
    ]
)
fig = px.histogram(
    predictions_df,
    x="Prediction",
    title="Prediction values",
    color="Label",
    marginal="box",
    labels={"0": "Legitimate", "1": "Fraudulent"},
)
fig.update_traces(opacity=0.75)
fig.update_layout(barmode="overlay")
fig.show()


787/787 [==============================] - 2s 2ms/step


In reality, we'd need to decide a threshold for a
transaction being marked as fraudulent. Rather than
working with this number, we can use a metric we actually care about. This is where something like the receiver operating characteristic (ROC) curve comes in.

The ROC curve maps the rate of true positives against the rate of false positives. This allows us to ask the question: "how many false positives are we willing to tolerate?" or "how many false negatives are we willing to tolerate?". In the context of fraud, this means either restricting a legitimate person's credit card, or letting a fraudster get away.

This ROC curve is one way to evaluate models. A perfect ROC curve starts at 100% true positives with 0% false negatives. In practice, models can't do this.

In [ ]:
def make_roc_df(name, predictions, labels):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)
    return pd.DataFrame({"fp": fp * 100, "tp": tp * 100, "Dataset": name})


roc_df = pd.concat(
    [
        make_roc_df("Training", train_predictions, train_labels),
        make_roc_df("Test", test_predictions, test_labels),
    ]
)

fig = px.line(
    roc_df,
    title="ROC Curve",
    x="fp",
    y="tp",
    color="Dataset",
    labels={"fp": "False Positives (%)", "tp": "True Positives (%)"},
)
fig.update_yaxes(range=[60, 100])
fig.update_traces(line={"dash": "dash"}, selector={"name": "test"})
fig.show()
